In [123]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


## Step 1 Initial Setup

In [125]:
import pulp

# Define the model
model = pulp.LpProblem("Amazon_Air_Distribution", pulp.LpMinimize)

## Step 2 Define Sets for Hubs, Focus Cities, and Centers

In [127]:
hubs = ['CVG', 'AFW']
focus = ['Leipzig', 'Hyderabad', 'SanBernardino']
centers = [f'C{k+1}' for k in range(65)]

## Step 3 Declare Decision Variables and Demand Data

In [129]:
x = pulp.LpVariable.dicts("x", [(i,j) for i in hubs for j in focus], lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts("y", [(i,k) for i in hubs for k in centers], lowBound=0, cat='Continuous')
z = pulp.LpVariable.dicts("z", [(j,k) for j in focus for k in centers], lowBound=0, cat='Continuous')

center_demand = {
    'C1': 6500, 'C2': 640, 'C3': 180, 'C4': 9100, 'C5': 570, 'C6': 19000, 'C7': 14800,
    'C8': 90, 'C9': 185, 'C10': 800, 'C11': 1700, 'C12': 170, 'C13': 2800, 'C14': 3700,
    'C15': 30, 'C16': 6700, 'C17': 190, 'C18': 175, 'C19': 38, 'C20': 2400,
    'C21': 7200, 'C22': 100, 'C23': 1200, 'C24': 1100, 'C25': 1900, 'C26': 240,
    'C27': 1500, 'C28': 540, 'C29': 3400, 'C30': 185, 'C31': 1600, 'C32': 3000,
    'C33': 500, 'C34': 16, 'C35': 63, 'C36': 5100, 'C37': 172, 'C38': 200, 'C39': 173,
    'C40': 300, 'C41': 290, 'C42': 550, 'C43': 1300, 'C44': 1700, 'C45': 975,
    'C46': 1200, 'C47': 480, 'C48': 100, 'C49': 450, 'C50': 11200, 'C51': 900,
    'C52': 290, 'C53': 150, 'C54': 1200, 'C55': 420, 'C56': 1000, 'C57': 1100,
    'C58': 650, 'C59': 975, 'C60': 3300, 'C61': 3300, 'C62': 1100, 'C63': 600,
    'C64': 2000, 'C65': 260
}
# Real-world cost data based on distribution chart

cost_y = {
    ('CVG', 'C1'): 1.6,
    ('CVG', 'C2'): 1.5,
    ('CVG', 'C3'): 1.5,
    ('CVG', 'C8'): 1.5,
    ('CVG', 'C9'): 1.5,
    ('CVG', 'C10'): 1.5,
    ('CVG', 'C11'): 1.5,
    ('CVG', 'C12'): 1.4,
    ('CVG', 'C13'): 1.5,
    ('CVG', 'C14'): 1.6,
    ('CVG', 'C15'): 1.4,
    ('CVG', 'C16'): 1.6,
    ('CVG', 'C17'): 0.5,
    ('AFW', 'C17'): 0.5,
    ('CVG', 'C18'): 1.3,
    ('AFW', 'C18'): 1,
    ('CVG', 'C19'): 1.4,
    ('AFW', 'C19'): 1,
    ('CVG', 'C20'): 0.5,
    ('AFW', 'C20'): 0.5
}

cost_z = {
    ('Leipzig', 'C1'): 0.5,
    ('Hyderabad', 'C1'): 1.1,
    ('Leipzig', 'C2'): 0.5,
    ('Hyderabad', 'C2'): 1,
    ('Leipzig', 'C3'): 0.5,
    ('Hyderabad', 'C3'): 1,
    ('Leipzig', 'C4'): 1.5,
    ('Hyderabad', 'C4'): 0.5,
    ('Leipzig', 'C5'): 1.5,
    ('Hyderabad', 'C5'): 0.5,
    ('Leipzig', 'C6'): 1.5,
    ('Hyderabad', 'C6'): 0.5,
    ('Leipzig', 'C7'): 1.5,
    ('Hyderabad', 'C7'): 0.5,
    ('Leipzig', 'C8'): 0.5,
    ('Hyderabad', 'C8'): 1,
    ('Leipzig', 'C9'): 0.5,
    ('Hyderabad', 'C9'): 1,
    ('Leipzig', 'C10'): 0.5,
    ('Hyderabad', 'C10'): 1,
    ('Leipzig', 'C11'): 0.5,
    ('Hyderabad', 'C11'): 1.1,
    ('Leipzig', 'C12'): 0.5,
    ('Hyderabad', 'C12'): 1,
    ('Leipzig', 'C13'): 0.5,
    ('Hyderabad', 'C13'): 1,
    ('Leipzig', 'C14'): 0.5,
    ('Hyderabad', 'C14'): 1.1,
    ('Leipzig', 'C15'): 0.5,
    ('Leipzig', 'C16'): 0.75,
    ('Hyderabad', 'C16'): 1.1,
    ('SanBernardino', 'C17'): 0.5,
    ('SanBernardino', 'C18'): 0.7,
    ('SanBernardino', 'C19'): 0.7,
    ('SanBernardino', 'C20'): 0.5
}

## Step 4 Input Cost Data

In [131]:
# Cost to send from hubs to focus cities
cost_x = {
    ('CVG', 'Leipzig'): 1.5,
    ('CVG', 'SanBernardino'): 0.5,
    ('AFW', 'SanBernardino'): 0.5
}

# Cost to send from hubs to centers
cost_y = {}
for i, center in enumerate(centers):
    cost_y[('CVG', center)] = 1.5 if i < 40 else None
    cost_y[('AFW', center)] = 0.5 if i < 30 else None

# Cost to send from focus cities to centers
cost_z = {}
for i, center in enumerate(centers):
    cost_z[('Leipzig', center)] = 0.5 if i < 40 else None
    cost_z[('Hyderabad', center)] = 1.1 if 3 <= i < 10 else None  # Indian cities
    cost_z[('SanBernardino', center)] = 0.5 if 15 <= i < 40 else None  # USA only

## Step 5 Define Objective Function

In [133]:
model += pulp.lpSum([
    cost_x[i, j] * x[i, j] for (i, j) in cost_x
]) + pulp.lpSum([
    cost_y[i, k] * y[i, k] for (i, k) in cost_y if cost_y[i, k] is not None
]) + pulp.lpSum([
    cost_z[j, k] * z[j, k] for (j, k) in cost_z if cost_z[j, k] is not None
]), "Total_Shipping_Cost"

## Step 6 Add Constraints

In [135]:
model += pulp.lpSum(
    [x['CVG', j] for j in focus if ('CVG', j) in x] +
    [y['CVG', k] for k in centers if ('CVG', k) in y]
) <= 95650, f"CVG_Capacity_{len(model.constraints)}"

model += pulp.lpSum(
    [x['AFW', j] for j in focus if ('AFW', j) in x] +
    [y['AFW', k] for k in centers if ('AFW', k) in y]
) <= 44350, f"AFW_Capacity_{len(model.constraints)}"

#Center Demand 
for k in centers:
    if k in center_demand:
        model += pulp.lpSum(
            [y[i, k] for i in hubs if (i, k) in y] +
            [z[j, k] for j in focus if (j, k) in z]
        ) == center_demand[k], f"Demand_{k}"
# Focus City Balance: Input from hubs = Output to centers
for j in focus:
    model += pulp.lpSum([x[i, j] for i in hubs if (i, j) in x]) == \
             pulp.lpSum([z[j, k] for k in centers if (j, k) in z]), f"FocusBalance_{j}"

## Step 7 Solve and Output Results

In [137]:
model.solve()

print("Status:", pulp.LpStatus[model.status])
print("Total Cost:", pulp.value(model.objective))

# Show only variables that were used
for var in model.variables():
    if var.varValue and var.varValue > 0:
        print(f"{var.name} = {var.varValue}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/trayvoniouspendleton/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/ll/cvqls06d7156l6ks5dqkj5940000gn/T/f98b999ce3b046cbb277a429ef176fa5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ll/cvqls06d7156l6ks5dqkj5940000gn/T/f98b999ce3b046cbb277a429ef176fa5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 75 COLUMNS
At line 883 RHS
At line 954 BOUNDS
At line 955 ENDATA
Problem MODEL has 70 rows, 331 columns and 662 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 70 (0) rows, 331 (0) columns and 662 (0) elements
Perturbing problem by 0.001% of 1.5 - largest nonzero change 1.2674376e-05 ( 0.0016324%) - largest zero change 5.7403951e-06
0  Obj 0 Primal inf 133747 (65)
73  Obj 22273.421
Optimal - objective value 22272.5
Optimal objective 2